# Data analysis

In [40]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.firefox import GeckoDriverManager
from urllib.parse import urlparse
import urllib
import json
import pandas as pd
import os
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - 

INFO:WDM:

[WDM] - ====== WebDriver manager ======
INFO:WDM:====== WebDriver manager ======
[WDM] - Current google-chrome version is 96.0.4664
INFO:WDM:Current google-chrome version is 96.0.4664
[WDM] - Get LATEST driver version for 96.0.4664
INFO:WDM:Get LATEST driver version for 96.0.4664
[WDM] - Driver [/home/marco/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache
INFO:WDM:Driver [/home/marco/.wdm/drivers/chromedriver/linux64/96.0.4664.45/chromedriver] found in cache
/tmp/ipykernel_1658359/2834769108.py:25: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [41]:
driver.get("https://sarc.asystom.com/monitor/login")
# import time
# time.sleep(1)
elem = driver.find_element_by_name("user")
elem.clear()
elem.send_keys(os.environ["SITE_USER"])

elem = driver.find_element_by_name("password")
elem.clear()
elem.send_keys(os.environ["SITE_PASSWORD"])

btn = driver.find_element_by_class_name("css-mij01a-button")
btn.click()

/tmp/ipykernel_1658359/3561746922.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_name("user")
/tmp/ipykernel_1658359/3561746922.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  elem = driver.find_element_by_name("password")
/tmp/ipykernel_1658359/3561746922.py:12: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  btn = driver.find_element_by_class_name("css-mij01a-button")


In [42]:
def get_dfs(parsed_json):
    dfs = []
    # parsed_json["results"][0]["series"][0]["columns"]
    # parsed_json["results"][0]["series"][0]["values"]

    for result in parsed_json["results"]:
        print("Result", result["statement_id"])
        for serie in result["series"]:
            df = pd.DataFrame(serie["values"], columns=serie["columns"])
            df.name = serie["name"]
            dfs.append(df)
    return dfs

In [25]:
from urllib import parse
q="SELECT%20median(%22drift%22)%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%2030d%20AND%20%22mileage%22%252%3D1)%20GROUP%20BY%20time(2h)%20fill(none)%3BSELECT%20moving_average(%22decimated%22%2C%2050)%20FROM%20(SELECT%20median(%22drift%22)%20AS%20%22decimated%22%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%2030d%20AND%20%22mileage%22%252%3D1)%20GROUP%20BY%20time(1m)%20fill(none))"

q="https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=SELECT%20last(%22drift%22)%20FROM%20(SELECT%2020*log10(%22vib_x_vel%22*%22vib_y_vel%22*%22vib_z_vel%22)%20AS%20%22drift%22%20FROM%20%22Signature_Drift%22%20WHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now()%20-%201d%20AND%20%22mileage%22%252%3D1)%0A&epoch=ms"
q_clean = parse.unquote(q)
q_clean

'https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=SELECT last("drift") FROM (SELECT 20*log10("vib_x_vel"*"vib_y_vel"*"vib_z_vel") AS "drift" FROM "Signature_Drift" WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - 1d AND "mileage"%2=1)\n&epoch=ms'

In [45]:
# SELECT vib_x_vel, vib_y_vel, vib_z_vel

delay = "200d"

q_clean = f"""
SELECT *
FROM "Signature_Drift"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};

SELECT *
FROM "Signature"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};


SELECT *
FROM "Temperature"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};


SELECT *
FROM "Current_Loop"
WHERE "device"=\'00-04-a3-0b-00-e9-b5-05\' AND time >= now() - {delay};
"""

driver.set_page_load_timeout(600) # 10 minutes

q_econded = parse.quote(q_clean)
url = f'https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q={q_econded}&epoch=ms'
print("Opening:", url)
driver.get(url)
driver.page_source
content = driver.find_element_by_tag_name('pre').text
parsed_json = json.loads(content)
parsed_json
dfs = get_dfs(parsed_json)

Opening: https://sarc.asystom.com/monitor/api/datasources/proxy/8/query?db=asystom_db&q=%0ASELECT%20%2A%0AFROM%20%22Signature_Drift%22%0AWHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now%28%29%20-%20200d%3B%0A%0ASELECT%20%2A%0AFROM%20%22Signature%22%0AWHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now%28%29%20-%20200d%3B%0A%0A%0ASELECT%20%2A%0AFROM%20%22Temperature%22%0AWHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now%28%29%20-%20200d%3B%0A%0A%0ASELECT%20%2A%0AFROM%20%22Current_Loop%22%0AWHERE%20%22device%22%3D%2700-04-a3-0b-00-e9-b5-05%27%20AND%20time%20%3E%3D%20now%28%29%20-%20200d%3B%0A&epoch=ms


/tmp/ipykernel_1658359/4058697289.py:32: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  content = driver.find_element_by_tag_name('pre').text


Result 0
Result 1
Result 2
Result 3


In [46]:
with open("all_data.json", "w") as f:
    json.dump(parsed_json, f)

In [ ]:
print(dfs[0].name)
dfs[0]

In [ ]:
print(dfs[1].name)
dfs[1]

Signature


,time,GW,client,device,mileage,s_00,s_01,s_02,s_03,s_04,...,vib_y_vel,vib_z_acc,vib_z_f1,vib_z_f2,vib_z_f3,vib_z_kurt,vib_z_peak,vib_z_root,vib_z_vel,vibra_custom
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-70.608835,-71.288624,-71.528954,-72.222477,-72.794690,...,0.108339,0.006348,0.0,0.0,0.0,3.065537,0.024659,0.0,0.090028,0
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-71.000229,-71.844816,-71.519799,-72.007324,-72.817578,...,0.076295,0.006104,0.0,0.0,0.0,3.060960,0.023438,0.0,0.076295,0
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-70.306706,-71.922637,-72.487984,-72.142367,-72.199588,...,0.093080,0.006104,0.0,0.0,0.0,3.042649,0.022950,0.0,0.100710,0
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-71.593042,-71.657130,-72.716869,-72.259098,-73.073930,...,0.105287,0.006104,0.0,0.0,0.0,2.975509,0.021729,0.0,0.073243,0
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0,-71.158160,-71.458000,-72.158389,-71.586175,-72.877089,...,0.065614,0.006104,0.0,0.0,0.0,2.957198,0.022461,0.0,0.094606,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13941,1636299812175,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-61.160449,-66.754406,-70.917830,-72.499428,-72.542916,...,0.384527,0.009766,0.0,0.0,0.0,2.835126,0.033204,0.0,0.321965,0
13942,1636299932138,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-61.007095,-67.219043,-71.924926,-72.657359,-72.540627,...,0.375372,0.009766,0.0,0.0,0.0,2.951095,0.039307,0.0,0.267033,0
13943,1636300051976,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-59.663539,-67.212177,-71.071183,-72.220188,-73.101396,...,0.387579,0.010010,0.0,0.0,0.0,3.108263,0.035401,0.0,0.363165,0
13944,1636300171969,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,512,-60.109865,-65.957885,-71.469444,-72.048524,-72.726024,...,0.398260,0.010254,0.0,0.0,0.0,2.851911,0.034424,0.0,0.361639,0


In [ ]:
print(dfs[2].name)
dfs[2]

Temperature


,time,GW,client,device,value,weekday
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.305737,Monday
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.305737,Monday
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,22.340423,Monday
...,...,...,...,...,...,...
17582,1636300051976,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,19.288051,Sunday
17583,1636300111833,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,19.253365,Sunday
17584,1636300171969,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,19.288051,Sunday
17585,1636300231821,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,19.288051,Sunday


In [ ]:
print(dfs[3].name)
dfs[3]

Current_Loop


,time,GW,client,device,value,weekday
0,1630308787973,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
1,1630308847953,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
2,1630308868190,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000916,Monday
3,1630308927926,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000916,Monday
4,1630308987909,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,0.000458,Monday
...,...,...,...,...,...,...
17582,1636300051976,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.949187,Sunday
17583,1636300111833,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.941405,Sunday
17584,1636300171969,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.943694,Sunday
17585,1636300231821,WMC_Sarc,WMC_Sarc,00-04-a3-0b-00-e9-b5-05,3.946899,Sunday
